In [9]:
# IP adress of this computer.
HOST = "192.168.1.100"
# Port on which the server will listen.
PORT = 1300
# Whether to preview the camera pictures.
SHOW_VIDEO = True
# ID of camera to use.
CAMERA_ID = 0

In [10]:
import json
import socketserver
import os
import base64
from io import BytesIO
from PIL import Image
import cv2
import numpy as np

In [11]:
cam = cv2.VideoCapture(CAMERA_ID)
cv2.namedWindow("Preview")
cv2.namedWindow("Mask")

_, image = cam.read()
#print(image.shape[:2])
size = image.shape
mask = np.ones(image.shape)
#print(mask.shape)

threshold = [0, 0, 0]
color_picked = np.zeros((3))

def on_change_threshold_h(value):
    threshold[0] = value
def on_change_threshold_s(value):
    threshold[1] = value
def on_change_threshold_v(value):
    threshold[2] = value

cv2.createTrackbar("threshold_h", "Preview", 0, 255, on_change_threshold_h)
cv2.createTrackbar("threshold_s", "Preview", 0, 255, on_change_threshold_s)
cv2.createTrackbar("threshold_v", "Preview", 0, 255, on_change_threshold_v)

def pickColor(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        _, image = cam.read()
        global color_picked
        color_picked = cv2.cvtColor(np.uint8([[image[y, x]]]), cv2.COLOR_BGR2HSV)[0][0]
        
        #image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        #global mask
        #mask = np.bitwise_and(mask, cv2.inRange(image_hsv, color_bound1, color_bound2).astype(dtype=bool))

cv2.setMouseCallback("Preview", pickColor)

In [12]:
while(SHOW_VIDEO):
    _, image = cam.read()
    
    image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    global color_picked
    color_bound1 = color_picked - threshold
    color_bound2 = color_picked + threshold
    mask = cv2.inRange(image_hsv, color_bound1, color_bound2)
    
    cv2.imshow("Preview", image)
    cv2.imshow("Mask", mask)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break

In [13]:
threshold = 120

def on_change_threshold(value):
    global threshold
    threshold = value

cv2.createTrackbar("threshold", "Result", 0, 255, on_change_threshold)
        
while(True):
    _, inv_image = cam.read()
    inv_image = inv_image[start_top:end_top, start_left:end_left]
    inv_masked_image = cv2.bitwise_and(inv_image, inv_image, mask=mask[start_top:end_top, start_left:end_left])
    masked_image = cv2.cvtColor(np.invert(inv_masked_image), cv2.COLOR_BGR2GRAY)
    masked_image = cv2.threshold(masked_image,threshold,255,cv2.THRESH_BINARY)[1]
    #print(threshold)
    cv2.imshow("Result", masked_image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break

120
120
120
120
120
120
120
120
120
120
120
120
120
120
120
120
120
120
120
120
120
120
120
120
120
120
120
120
120
120
120


In [14]:
class TCPHandler(socketserver.StreamRequestHandler):

    def handle(self):
        print("client connected.")
        try:
            cam = cv2.VideoCapture(CAMERA_ID)
            _, raw_image = cam.read()
            raw_image = raw_image[start_top:end_top, start_left:end_left]
            inv_masked_image = cv2.bitwise_and(raw_image, raw_image, mask=mask[start_top:end_top, start_left:end_left])
            header = json.loads(self.rfile.readline().strip())["header"]
            if (header == "image"):
                _, image_arr = cv2.imencode('.jpg', inv_masked_image)
                image_bytes = image_arr.tobytes()
                image_b64 = base64.b64encode(image_bytes)
                self.wfile.write(image_b64)
            elif(json.loads(self.rfile.readline().strip())["header"] == "raw"):
                _, image_arr = cv2.imencode('.jpg', raw_image)
                image_bytes = image_arr.tobytes()
                image_b64 = base64.b64encode(image_bytes)
                self.wfile.write(image_b64)
            else:
                masked_image = cv2.cvtColor(np.invert(inv_masked_image), cv2.COLOR_BGR2GRAY)
                masked_image = cv2.threshold(masked_image,threshold,255,cv2.THRESH_BINARY)[1]
                _, image_arr = cv2.imencode('.jpg', masked_image)
                image_bytes = image_arr.tobytes()
                image_b64 = base64.b64encode(image_bytes)
                
                self.wfile.write(image_b64)
        except Exception as e:
            print(e.message, e.args)

with socketserver.TCPServer((HOST, PORT), TCPHandler) as server:
    print("Starting server.")
    server.serve_forever()

Starting server.
client connected.


----------------------------------------
Exception occurred during processing of request from ('192.168.1.104', 45640)
Traceback (most recent call last):
  File "C:\Users\benja\AppData\Local\Temp\ipykernel_22452\69933158.py", line 14, in handle
    elif(json.loads(self.rfile.readline().strip())["header"] == "image"):
  File "c:\Users\benja\.conda\envs\test\lib\json\__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "c:\Users\benja\.conda\envs\test\lib\json\decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "c:\Users\benja\.conda\envs\test\lib\json\decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\benja\.conda\envs\test\lib\socketserver.py", line 316, in _handle_request_no

client connected.


----------------------------------------
Exception occurred during processing of request from ('192.168.1.104', 45654)
Traceback (most recent call last):
  File "C:\Users\benja\AppData\Local\Temp\ipykernel_22452\69933158.py", line 14, in handle
    elif(json.loads(self.rfile.readline().strip())["header"] == "image"):
  File "c:\Users\benja\.conda\envs\test\lib\json\__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "c:\Users\benja\.conda\envs\test\lib\json\decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "c:\Users\benja\.conda\envs\test\lib\json\decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\benja\.conda\envs\test\lib\socketserver.py", line 316, in _handle_request_no

client connected.


...

client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client con

client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client con

client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client con

client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client connected.
client con

KeyboardInterrupt: 